<a href="https://colab.research.google.com/github/Joycechidi/Secure-and-Private-AI/blob/master/encrypted_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Section: Encrypted Deep Learning**

**Reviewing Additive Secret Sharing**

In [0]:
import random
import numpy as np


BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)


def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element


def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

def encrypt(secret):
    first = random.randrange(Q)
    second = random.randrange(Q)
    third = (secret - first - second) % Q
    return [first, second, third]

def decrypt(sharing):
    return sum(sharing) % Q

def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)


In [0]:
x = encrypt(encode(5.5))
x

[90283571305293152171353630155425151236,
 96830848460514787209495369179516930861,
 106858925709359307689596278445973662316]

In [0]:
y = encrypt(encode(2.3))
y

[240740920906344942757601672581998907591,
 267807806849171704366072275995559076954,
 79397963194817847017216606983403504280]

In [0]:
z = add(x, y)
z

(37051146736470847858510024957058314414,
 70665309834519244505122367394710263402,
 186256888904177154706812885429377166596)

In [0]:
decode(decrypt(z))

7.79999999

# Build an Encrypted Database

In [0]:
field = 23740629843760239486723

# Encrypted Deep Learning in PyTorch

## Train a Model 

In [0]:
!pip install syft

     |████████████████████████████████| 215kB 4.8MB/s 
     |████████████████████████████████| 1.4MB 48.6MB/s 
     |████████████████████████████████| 450kB 46.6MB/s 
     |████████████████████████████████| 71kB 23.2MB/s 
     |████████████████████████████████| 204kB 37.8MB/s 
     |████████████████████████████████| 256kB 46.4MB/s 
     |████████████████████████████████| 389kB 41.6MB/s 
     |████████████████████████████████| 276kB 47.3MB/s 
     |████████████████████████████████| 51kB 19.3MB/s 
     |████████████████████████████████| 122kB 44.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [0]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim
import torch.nn.functional as F

W0703 01:34:44.412805 140549417944960 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0703 01:34:44.432689 140549417944960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [0]:
chidi = sy.VirtualWorker(hook, id="chidi").add_worker(sy.local_worker)
ify = sy.VirtualWorker(hook, id="ify").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [0]:
x = x.share(chidi, ify, crypto_provider=secure_worker)
y = y.share(chidi, ify, crypto_provider=secure_worker)

In [0]:
z = x + y
z.get()

tensor([3, 1, 4, 4])

In [0]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [0]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])